In [ ]:
import sys
import os

import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import tqdm
%pylab inline

import constants

In [ ]:
BIRTH_DIR = '/global/homes/b/bzh/clamato-xcorr/data/birth'

os.chdir(BIRTH_DIR)
import coord_conversion

In [ ]:
CLAMATO_FOOTPRINT_RA = (149.89 * u.deg, 150.4 * u.deg)
CLAMATO_FOOTPRINT_DEC = (2.09 * u.deg, 2.5 * u.deg)
FOOTPRINT_PATH = 'clamato_footprint_mask.npy'

if os.path.exists(FOOTPRINT_PATH):
    clamato_footprint_mask = np.load(FOOTPRINT_PATH)
else:
    clamato_footprint_mask = np.zeros_like(coord_conversion.sm_delta)

    indices = []
    for x in range(0, coord_conversion.sm_delta.shape[0], 3):
        for y in range(0, coord_conversion.sm_delta.shape[1], 3):
            for z in range(0, coord_conversion.sm_delta.shape[2], 3):
                indices.append(((x + 1), (y + 1), (z + 1)))

    # indices = [(x, y, z) for x in range(coord_conversion.sm_delta.shape[0]) for y in range(coord_conversion.sm_delta.shape[1]) for z in range(coord_conversion.sm_delta.shape[2])]

    for ind in tqdm.tqdm(indices):
        x, y, z = ind
        coord = coord_conversion.original_pos(*coord_conversion.index2coord(x, y, z))
        if (not coord.ra) or (not coord.dec):
            print(f'Coordinate for ({x}, {y}, {z}), does not have RA or DEC; skipping.')
            continue
        if (2.05 <= coord.distance <= 2.55
            and CLAMATO_FOOTPRINT_RA[0] <= coord.ra <= CLAMATO_FOOTPRINT_RA[1] 
            and CLAMATO_FOOTPRINT_DEC[0] <= coord.dec <= CLAMATO_FOOTPRINT_DEC[1]):
            clamato_footprint_mask[x-1:x+2, y-1:y+2, z-1:z+2] = 1
    np.save(FOOTPRINT_PATH, clamato_footprint_mask)
                
print(f'Mask covers {np.sum(clamato_footprint_mask) / clamato_footprint_mask.size * 100}% of volume.')

In [ ]:
plt.figure(figsize=(10,10))
z = 126
plt.imshow(clamato_footprint_mask[z, :, :])

In [ ]:
redshift_mask = np.where(clamato_footprint_mask[:, 128, 128])[0]
print(len(redshift_mask))

In [ ]:
stamp_size = (np.argwhere(clamato_footprint_mask[126, :, 128]).size, np.argwhere(clamato_footprint_mask[126, 128, :]).size)
print(stamp_size)

In [ ]:
print(np.argwhere(clamato_footprint_mask[126, :, 128]))
print(np.argwhere(clamato_footprint_mask[126, 128, :]))

In [ ]:
plt.imshow(clamato_footprint_mask[z, :, :])
plt.axvline(123, color='red')
plt.axhline(120, color='red')

In [ ]:
clamato_footprint_sm_delta = np.mean(coord_conversion.sm_delta[clamato_footprint_mask.astype(np.bool)])
# volume_footprint_sm_delta = np.mean(coord_conversion.sm_delta[~clamato_footprint_mask.astype(np.bool)])
print(clamato_footprint_sm_delta)
# print(volume_footprint_sm_delta)

In [ ]:
bootstrap_iter = 100000
null_means = []
max_stamp = None
max_mean = -np.inf
np.random.seed(42)
for _ in tqdm.tqdm(range(bootstrap_iter)):
    stamp_corner = (np.random.randint(0, coord_conversion.sm_delta.shape[1] - stamp_size[0]), np.random.randint(0, coord_conversion.sm_delta.shape[2] - stamp_size[1]))
    null_means.append(np.mean(coord_conversion.sm_delta[redshift_mask, stamp_corner[0]:stamp_corner[0] + stamp_size[0], stamp_corner[1]:stamp_corner[1] + stamp_size[1]]))
    if null_means[-1] > max_mean:
        max_stamp = stamp_corner
        max_mean = null_means[-1]

In [ ]:
plt.hist(null_means, bins=20, color='black');
plt.axvline(clamato_footprint_sm_delta, color='red')
print((clamato_footprint_sm_delta - np.mean(null_means)) / np.std(null_means))

In [ ]:
print(max_stamp, max_mean)

In [ ]:
def cut_out_stamp(stamp):
    max_stamp_sm_delta = np.copy(coord_conversion.sm_delta)
    mask = np.zeros_like(max_stamp_sm_delta).astype(bool)
    mask[redshift_mask, stamp[0]:stamp[0] + stamp_size[0], stamp[1]:stamp[1] + stamp_size[1]] = True
    return max_stamp_sm_delta[mask].reshape(len(redshift_mask), *stamp_size)

# print(max_stamp_sm_delta.shape)
fig, axes = plt.subplots(1, 2, figsize=(2, 10))
axes[0].imshow(cut_out_stamp(max_stamp)[:, :, 7])
axes[1].imshow(cut_out_stamp((120, 123))[:, :, 7])

In [ ]:
mask.shape